In [1]:
import pandas as pd
data = pd.read_csv("train.csv")
submit = pd.read_csv('sample_submission.csv')

In [2]:
import pandas as pd
from datetime import datetime
from darts import TimeSeries
from darts.models import NLinearModel


# 데이터를 불러오기

df = pd.DataFrame(data)
df['일시'] = pd.to_datetime(df['일시'])

# NaN 값 처리 (예시로 평균값으로 대체)
df = df.fillna(df.mean())

# TimeSeries 객체 생성
series = TimeSeries.from_dataframe(df, '일시', ['평균기온'])


# NLinearModel 모델 생성 및 훈련
model = NLinearModel(input_chunk_length=709, 
                     output_chunk_length=460, 
                     n_epochs=1,
                     random_state = 42,
                     batch_size = 293)
model.fit(series)

D:\mdrive\dacon\envs\sun\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")
C:\Users\82105\AppData\Local\Temp\ipykernel_13952\4091972756.py:13: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df = df.fillna(df.mean())
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | layer         | Linear           | 326 K 
---------------------------------------------------
326 K     Trainable params
0         Non-tr

Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=1` reached.


NLinearModel(shared_weights=False, const_init=True, normalize=False, use_static_covariates=True, input_chunk_length=709, output_chunk_length=460, n_epochs=1, random_state=42, batch_size=293)

In [3]:
future = pd.DataFrame()
future['일시'] = pd.date_range(start='2023-01-01', periods=358, freq='D')

# 모델로 예측 수행
forecast = model.predict(len(future['일시']))
pred_y = forecast['평균기온']

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

In [4]:
pred_y['평균기온'].values

<bound method TimeSeries.values of <TimeSeries (DataArray) (일시: 358, component: 1, sample: 1)>
array([[[-1.97777815]],

       [[-1.8474555 ]],

       [[-1.87907656]],

       [[-2.15782675]],

       [[-2.04946187]],

       [[-1.69347758]],

       [[-0.64261006]],

       [[-0.47493156]],

       [[-1.44743176]],

       [[-0.85983496]],

...

       [[ 1.70681223]],

       [[ 1.27904887]],

       [[ 0.15077911]],

       [[ 0.24764054]],

       [[ 0.23438581]],

       [[ 0.04056526]],

       [[-0.3453204 ]],

       [[-0.84971934]],

       [[-0.68392419]],

       [[ 0.55931696]]])
Coordinates:
  * 일시         (일시) datetime64[ns] 2023-01-01 2023-01-02 ... 2023-12-24
  * component  (component) object '평균기온'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None>

In [5]:
import pandas as pd

# Assuming pred_y is a TimeSeries object
pred_y_values = pred_y.pd_dataframe().values

# Now you can use pred_y_values as a numpy array or convert it to a DataFrame
df_pred_y = pd.DataFrame(pred_y_values, index=pred_y.time_index, columns=pred_y.columns)


In [6]:
submit['평균기온'] = df_pred_y['평균기온'].values

In [7]:
#결과 저장
submit.to_csv("FFFFFINAL.csv", index=False)

In [8]:
submit

,일시,평균기온
0,2023-01-01,-1.977778
1,2023-01-02,-1.847456
2,2023-01-03,-1.879077
3,2023-01-04,-2.157827
4,2023-01-05,-2.049462
...,...,...
353,2023-12-20,0.040565
354,2023-12-21,-0.345320
355,2023-12-22,-0.849719
356,2023-12-23,-0.683924
